In [0]:
confluentClusterName = "kafka-with-databricks"
confluentBootstrapServers = "pkc-lgwgm.eastus2.azure.confluent.cloud:9092"
confluentTopicName = "orders"
confluentApiKey = "EFT7XS6PO2554TVM"
confluentSecret = "cflt4+/pWRCJXakCBUujthgjiCk+/K6lNvoN22um6DEtZu2a6/KwtsYweSprnjtw"
schemaRegistryUrl = "https://psrc-8qyy0.eastus2.azure.confluent.cloud"
confluentRegistryApiKey = "PJ2IJQFJ35AVY2BJ"
confluentRegistrySecret = "cflt+Zh4iEzxbfy7aSnTDYVOegH3bqp8ARedoMUPbmcfkb0gYLoJj6GuEVe3Ntig"
deltaTablePath = "/Volumes/demo_catalog_for_streaming_data/data-from-kafka/raw_data/orders"
checkpointPath = "/Volumes/demo_catalog_for_streaming_data/data-from-kafka/raw_data/checkpoint/orders"

In [0]:
from confluent_kafka.schema_registry import SchemaRegistryClient
import ssl

schema_registry_conf = {
    "url": schemaRegistryUrl,
    "basic.auth.user.info": "{}:{}".format(
        confluentRegistryApiKey, confluentRegistrySecret
    ),
}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [0]:
import pyspark.sql.functions as fn
from pyspark.sql.types import StringType

binary_to_string = fn.udf(
    lambda x: str(int.from_bytes(x, byteorder="big")), StringType()
)
streamTestDf = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", confluentBootstrapServers)
    .option("kafka.security.protocol", "SASL_SSL")
    .option(
        "kafka.sasl.jaas.config",
        "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(
            confluentApiKey, confluentSecret
        ),
    )
    .option("kafka.ssl.endpoint.identification.algorithm", "https")
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("subscribe", confluentTopicName)
    .option("startingOffsets", "earliest")
    .option("failOnDataLoss", "false")
    .load()
    .withColumn("key", fn.col("key").cast(StringType()))
    .withColumn("fixedValue", fn.expr("substring(value, 6, length(value)-5)"))
    .withColumn("valueSchemaId", binary_to_string(fn.expr("substring(value, 2, 4)")))
    .select(
        "topic",
        "partition",
        "offset",
        "timestamp",
        "timestampType",
        "key",
        "valueSchemaId",
        "fixedValue",
    )
)

In [0]:
streamTestDf.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/demo_catalog_for_streaming_data/data-from-kafka/raw_data") \
    .option("path", "/Volumes/demo_catalog_for_streaming_data/data-from-kafka/raw_data") \
    .outputMode("append") \
    .trigger(availableNow=True) \
    .start()

In [0]:
import pyspark.sql.functions as fn
from pyspark.sql.avro.functions import from_avro


def parseAvroDataWithSchemaId(df, ephoch_id):
    cachedDf = df.cache()
    fromAvroOptions = {"mode": "FAILFAST"}

    def getSchema(id):
        return str(schema_registry_client.get_schema(id).schema_str)

    distinctValueSchemaIdDF = cachedDf.select(
        fn.col("valueSchemaId").cast("integer")
    ).distinct()
    for valueRow in distinctValueSchemaIdDF.collect():
        currentValueSchemaId = sc.broadcast(valueRow.valueSchemaId)
        currentValueSchema = sc.broadcast(getSchema(currentValueSchemaId.value))
        filterValueDF = cachedDf.filter(
            fn.col("valueSchemaId") == currentValueSchemaId.value
        )
        filterValueDF.select(
            "topic",
            "partition",
            "offset",
            "timestamp",
            "timestampType",
            "key",
            from_avro("fixedValue", currentValueSchema.value, fromAvroOptions).alias(
                "parsedValue"
            ),
        ).write.format("delta").mode("append").option("mergeSchema", "true").save(
            deltaTablePath
        )

In [0]:
def parseAvroDataWithSchemaId(
    batch_df,
    batch_id
):
    # Access Spark session only via batch_df.sparkSession
    # All processing should use batch_df and batch_id
    pass

streamTestDf.writeStream \
    .option("checkpointLocation", checkpointPath) \
    .foreachBatch(parseAvroDataWithSchemaId) \
    .queryName("clickStreamTestFromConfluent") \
    .trigger(availableNow=True) \
    .start()

In [0]:
deltaTestDf = spark.read.format("delta").load(deltaTablePath)
display(deltaTestDf)

In [0]:
deltaTestDf.count()

In [0]:
deltaTestDf.printSchema()

In [0]:
from pyspark.sql.functions import col

# Assuming your DataFrame is named 'df'
new_df = deltaTestDf.select(
    col('topic'),
    col('partition'),
    col('offset'),
    col('timestamp'),
    col('timestampType'),
    col('key'),
    col('parsedValue.ordertime').alias('ordertime'),
    col('parsedValue.orderid').alias('orderid'),
    col('parsedValue.itemid').alias('itemid'),
    col('parsedValue.orderunits').alias('orderunits'),
    col('parsedValue.address.city').alias('city'),
    col('parsedValue.address.state').alias('state'),
    col('parsedValue.address.zipcode').alias('zipcode')
)

# Now 'new_df' contains exploded columns from 'parsedValue'
new_df.display()